**A. Read Tables from Catalog**

We are going to assume that we receive a full file everyday for furniture, office supplies, and technology.
We apply the same transformations on them, so we will parameterize the table name


In [0]:
dbutils.widgets.dropdown('category', 'furniture', ['furniture', 'office_supplies', 'technology'])

In [0]:
category = dbutils.widgets.get('category')

In [0]:
df = spark.table(f"mike_dbricks.superstore.{category}")
display(df.limit(5))

**B. Do some transformations**

In [0]:
from pyspark.sql.functions import col, expr
## Explore two ways to compute on a field for new derived fields


df = df.withColumn("cost_of_production", col("Sales") - col("Profit"))
df = df.select("*", expr("Sales - Profit AS cost_per_unit"))
display(df.select("cost_of_production", "cost_per_unit"))

In [0]:
from pyspark.sql.functions import col

# Rename columns to remove spaces and special characters
rename_dict = {
    "Row ID": "Row_ID",
    "Order ID": "Order_ID",
    "Order Date": "Order_Date",
    "Ship Date": "Ship_Date",
    "Ship Mode": "Ship_Mode",
    "Customer ID": "Customer_ID",
    "Customer Name": "Customer_Name",
    "Postal Code": "Postal_Code",
    "Product ID": "Product_ID",
    "Product Name": "Product_Name"
}

for old_col, new_col in rename_dict.items():
    df = df.withColumnRenamed(old_col, new_col)


In [0]:
df.write.format("delta").mode("overwrite").option("optimizeWrite", "true").saveAsTable(f"mike_dbricks.superstore.{category}_processed")

In [0]:
dbutils.notebook.exit("success")